<a href="https://colab.research.google.com/github/tabaahi07/Deep-Learning/blob/main/gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
from sklearn.model_selection import train_test_split
xtrain , xtest , ytrain , ytest = train_test_split(df[['age' , 'affordibility']] , df[['bought_insurance']] , test_size=0.3)
xtrain_scaled = xtrain.copy()
xtest_scaled = xtest.copy()
xtrain_scaled['age'] = xtrain_scaled['age'] / 100
xtest_scaled['age'] = xtest_scaled['age'] / 100
xtrain_scaled.head()

,age,affordibility
7,0.60,0
27,0.46,1
26,0.23,1
0,0.22,1
5,0.56,1


In [8]:
# Initially we'll set weights = 1 and bias=0 and then we'll backtrack and adjust weights to get minimum loss
# output , input_neurons=2 , initial_weights = 1   ,   b=0
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,) , activation='sigmoid' , kernel_initializer='ones' , bias_initializer='zeros')
])
model.compile(
    optimizer = 'adam' ,
    loss = 'binary_crossentropy' ,
    metrics = ['accuracy']
)
model.fit(xtrain_scaled , ytrain , epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 607ms/step - loss: 0.8283 - accuracy: 0.4211
Epoch 2/500
1/1 [==============================] - 0s 14ms/step - loss: 0.8277 - accuracy: 0.4211
Epoch 3/500
1/1 [==============================] - 0s 13ms/step - loss: 0.8271 - accuracy: 0.4211
Epoch 4/500
1/1 [==============================] - 0s 24ms/step - loss: 0.8266 - accuracy: 0.4211
Epoch 5/500
1/1 [==============================] - 0s 23ms/step - loss: 0.8260 - accuracy: 0.4211
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 0.8254 - accuracy: 0.4211
Epoch 7/500
1/1 [==============================] - 0s 18ms/step - loss: 0.8249 - accuracy: 0.4211
Epoch 8/500
1/1 [==============================] - 0s 15ms/step - loss: 0.8243 - accuracy: 0.4211
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 0.8238 - accuracy: 0.4211
Epoch 10/500
1/1 [==============================] - 0s 14ms/step - loss: 0.8232 - accuracy: 0.4211
Epoch 11/500
1/1 [

In [9]:
model.evaluate(xtest_scaled , ytest)

1/1 [==============================] - 0s 310ms/step - loss: 0.5352 - accuracy: 0.8889


[0.5352462530136108, 0.8888888955116272]

In [10]:
xtest_scaled

,age,affordibility
25,0.54,1
1,0.25,0
17,0.58,1
23,0.45,1
21,0.26,0
10,0.18,1
14,0.49,1
24,0.50,1
2,0.47,1


In [11]:
model.predict(xtest_scaled)

1/1 [==============================] - 0s 183ms/step


array([[0.6317587 ],
       [0.4376957 ],
       [0.6378923 ],
       [0.61780393],
       [0.43932435],
       [0.5748438 ],
       [0.6240316 ],
       [0.6255822 ],
       [0.6209227 ]], dtype=float32)

In [12]:
# To get the final weights after model has trained
wts , bias = model.get_weights()
wts , bias

(array([[0.66146517],
        [0.5984635 ]], dtype=float32),
 array([-0.41588545], dtype=float32))

In [13]:
# Sigmoid Function
def sigmoid(x) :
  return 1 / (1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [14]:
# Prediction Function
def prediction_fun(age , affordability) :
  # w1*age + w2*affordability + bias
  weighted_sum = wts[0]*age + wts[1]*affordability + bias
  return sigmoid(weighted_sum)

In [15]:
# Now we'll calc the value by prediction func and match it with predicted value by model
prediction_fun(0.55,0)

0.4869830449626738

In [16]:
# Log loss Function
def log_loss(y_true , y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean( y_true * np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))

In [17]:
def sigmoid(X) :
  return 1 / (1+np.exp(-X))

sigmoid(np.array([8,0,1]))

array([0.99966465, 0.5       , 0.73105858])

In [18]:
def gradient_descent(age , affordability , y_true , epochs ) :
  w1 = w2 = 1
  bias = 0
  learning_rate = 0.5
  n = len(age)

  for i in range(epochs) :
    weighted_sum = w1*age + w2*affordability + bias
    y_predicted = sigmoid(weighted_sum)
    loss = log_loss(y_true , y_predicted)

    # Now we'll adjust weights so calc d/dw1 and d/dw2 and d/d_bias
    # so new_w1 = w1 - learning_rate * d/dw1 , simimlarly new_w2
    w1d = (1/n) * np.dot( np.transpose(age) , (y_predicted-y_true) )
    w2d = (1/n) * np.dot( np.transpose(affordability) , (y_predicted-y_true) )
    bias_d = np.mean(y_predicted-y_true)

    w1 = w1 - learning_rate * w1d
    w2 = w2 - learning_rate * w2d
    bias = bias - learning_rate * bias_d

    print(f'Epoch : {i} , w1 : {w1} , w2 : {w2} , bias : {bias} , loss : {loss}')
  return w1 , w2 , bias

In [ ]:
gradient_descent(np.array(xtrain_scaled['age']) , np.array(xtrain_scaled['affordibility']) , np.array(ytrain) , 100)